<a href="https://colab.research.google.com/github/mvkumar14/DS-Unit-2-Linear-Models/blob/master/module3-ridge-regression/LS_DS_213_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html). Use the scaler's `fit_transform` method with the train set. Use the scaler's `transform` method with the test set.
- [ ] Fit a ridge regression model with multiple features.
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [14]:
condition = ((df.BUILDING_CLASS_CATEGORY == '01 ONE FAMILY DWELLINGS') & 
            (df.SALE_PRICE > 100000) & 
            (df.SALE_PRICE < 2000000))
df = df[condition]
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,NaN,A9,4832 BAY PARKWAY,NaN,11230.0,1.0,0.0,1.0,"6,800",1325.0,1930.0,1,A9,550000,01/01/2019
61,4,OTHER,01 ONE FAMILY DWELLINGS,1,7918,72,NaN,A1,80-23 232ND STREET,NaN,11427.0,1.0,0.0,1.0,"4,000",2001.0,1940.0,1,A1,200000,01/01/2019
78,2,OTHER,01 ONE FAMILY DWELLINGS,1,4210,19,NaN,A1,1260 RHINELANDER AVE,NaN,10461.0,1.0,0.0,1.0,"3,500",2043.0,1925.0,1,A1,810000,01/02/2019
108,3,OTHER,01 ONE FAMILY DWELLINGS,1,5212,69,NaN,A1,469 E 25TH ST,NaN,11226.0,1.0,0.0,1.0,"4,000",2680.0,1899.0,1,A1,125000,01/02/2019
111,3,OTHER,01 ONE FAMILY DWELLINGS,1,7930,121,NaN,A5,5521 WHITTY LANE,NaN,11203.0,1.0,0.0,1.0,"1,710",1872.0,1940.0,1,A5,620000,01/02/2019


In [34]:
import datetime
test= pd.to_datetime(df.SALE_DATE,infer_datetime_format=True)
test.iloc[1]

print(test.iloc[1]>test_date)
print(test.iloc[1],test_date)

True
2019-01-01 00:00:00 2018-04-01


In [125]:
df['SALE_DATE'] = pd.to_datetime(df.SALE_DATE)
cutoff_date = datetime.date(2019,4,1)
train = df[df.SALE_DATE < cutoff_date]
test = df[df.SALE_DATE >= cutoff_date]
print(test.shape)
print(train.shape)

(644, 21)
(2507, 21)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  after removing the cwd from sys.path.


In [126]:
train.select_dtypes(exclude='number')

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,LAND_SQUARE_FEET,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_DATE
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,A9,4832 BAY PARKWAY,NaN,"6,800",A9,2019-01-01
61,4,OTHER,01 ONE FAMILY DWELLINGS,1,A1,80-23 232ND STREET,NaN,"4,000",A1,2019-01-01
78,2,OTHER,01 ONE FAMILY DWELLINGS,1,A1,1260 RHINELANDER AVE,NaN,"3,500",A1,2019-01-02
108,3,OTHER,01 ONE FAMILY DWELLINGS,1,A1,469 E 25TH ST,NaN,"4,000",A1,2019-01-02
111,3,OTHER,01 ONE FAMILY DWELLINGS,1,A5,5521 WHITTY LANE,NaN,"1,710",A5,2019-01-02
...,...,...,...,...,...,...,...,...,...,...
18129,5,OTHER,01 ONE FAMILY DWELLINGS,1,A2,10 SEAFOAM STREET,NaN,"2,400",A2,2019-03-29
18130,5,OTHER,01 ONE FAMILY DWELLINGS,1,A5,74 MCVEIGH AVE,NaN,"2,450",A5,2019-03-29
18132,5,OTHER,01 ONE FAMILY DWELLINGS,1,A1,479 VILLA AVENUE,NaN,"4,361",A1,2019-03-29
18134,5,OTHER,01 ONE FAMILY DWELLINGS,1,A2,63 NUGENT AVENUE,NaN,"6,000",A2,2019-03-29


In [127]:
#dunno why this is a string, but not quite something to change right now.
type(train.LAND_SQUARE_FEET.iloc[1])

str

In [128]:
#I need to get rid of Easement
train.describe()

,BLOCK,LOT,EASE-MENT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,SALE_PRICE
count,2507.000000,2507.000000,0.0,2507.000000,2507.000000,2507.000000,2507.000000,2507.000000,2507.000000,2507.0,2.507000e+03
mean,6758.303949,75.778221,NaN,10993.398484,0.987635,0.016354,1.003989,1473.744715,1944.766653,1.0,6.215737e+05
std,3975.909029,157.531138,NaN,494.291462,0.110532,0.129966,0.171794,599.217635,27.059337,0.0,2.916072e+05
min,21.000000,1.000000,NaN,10301.000000,0.000000,0.000000,0.000000,0.000000,1890.000000,1.0,1.040000e+05
25%,3837.500000,21.000000,NaN,10314.000000,1.000000,0.000000,1.000000,1144.000000,1925.000000,1.0,4.405000e+05
50%,6022.000000,42.000000,NaN,11234.000000,1.000000,0.000000,1.000000,1368.000000,1940.000000,1.0,5.600000e+05
75%,9888.500000,70.000000,NaN,11413.000000,1.000000,0.000000,1.000000,1683.000000,1960.000000,1.0,7.500000e+05
max,16350.000000,2720.000000,NaN,11697.000000,1.000000,2.000000,3.000000,7875.000000,2018.000000,1.0,1.955000e+06


In [129]:
train.select_dtypes(exclude='number').describe()
#I'm going to get rid of :
# unselected = ['SALE_DATE','LAND_SQUARE_FEET','ADDRESS']

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,LAND_SQUARE_FEET,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_DATE
count,2507,2507,2507,2507,2507,2507,1,2507,2507,2507
unique,5,7,1,2,13,2497,1,887,11,68
top,4,OTHER,01 ONE FAMILY DWELLINGS,1,A1,130-52 LEFFERTS BOULEVARD,RP.,"4,000",A1,2019-01-31 00:00:00
freq,1204,2360,2507,2476,919,2,1,234,919,78
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-01 00:00:00
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-03-30 00:00:00


In [130]:
train.columns

Index(['BOROUGH', 'NEIGHBORHOOD', 'BUILDING_CLASS_CATEGORY',
       'TAX_CLASS_AT_PRESENT', 'BLOCK', 'LOT', 'EASE-MENT',
       'BUILDING_CLASS_AT_PRESENT', 'ADDRESS', 'APARTMENT_NUMBER', 'ZIP_CODE',
       'RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS', 'TOTAL_UNITS',
       'LAND_SQUARE_FEET', 'GROSS_SQUARE_FEET', 'YEAR_BUILT',
       'TAX_CLASS_AT_TIME_OF_SALE', 'BUILDING_CLASS_AT_TIME_OF_SALE',
       'SALE_PRICE', 'SALE_DATE'],
      dtype='object')

In [203]:
target = 'SALE_PRICE'
#high cardinality / non useful categories
other = ['EASE-MENT']
high_cardinality = ['SALE_DATE','ADDRESS','APARTMENT_NUMBER','BUILDING_CLASS_CATEGORY','LAND_SQUARE_FEET']
features = train.columns.drop([target]+high_cardinality+other)
features

Index(['BOROUGH', 'NEIGHBORHOOD', 'TAX_CLASS_AT_PRESENT', 'BLOCK', 'LOT',
       'BUILDING_CLASS_AT_PRESENT', 'ZIP_CODE', 'RESIDENTIAL_UNITS',
       'COMMERCIAL_UNITS', 'TOTAL_UNITS', 'GROSS_SQUARE_FEET', 'YEAR_BUILT',
       'TAX_CLASS_AT_TIME_OF_SALE', 'BUILDING_CLASS_AT_TIME_OF_SALE'],
      dtype='object')

In [204]:
X_train

,BOROUGH,NEIGHBORHOOD,TAX_CLASS_AT_PRESENT,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE
44,3,OTHER,1,5495,801,A9,11230.0,1.0,0.0,1.0,1325.0,1930.0,1,A9
61,4,OTHER,1,7918,72,A1,11427.0,1.0,0.0,1.0,2001.0,1940.0,1,A1
78,2,OTHER,1,4210,19,A1,10461.0,1.0,0.0,1.0,2043.0,1925.0,1,A1
108,3,OTHER,1,5212,69,A1,11226.0,1.0,0.0,1.0,2680.0,1899.0,1,A1
111,3,OTHER,1,7930,121,A5,11203.0,1.0,0.0,1.0,1872.0,1940.0,1,A5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18129,5,OTHER,1,4081,44,A2,10306.0,1.0,0.0,1.0,921.0,1950.0,1,A2
18130,5,OTHER,1,2373,201,A5,10314.0,1.0,0.0,1.0,2128.0,1980.0,1,A5
18132,5,OTHER,1,1132,42,A1,10302.0,1.0,0.0,1.0,1807.0,2018.0,1,A1
18134,5,OTHER,1,3395,37,A2,10305.0,1.0,0.0,1.0,621.0,1930.0,1,A2


In [0]:
#Set train and test values
X_train = train[features]
y_train = train[target]

X_test = test[features]
y_test = test[target]

In [206]:
#The variables to one-hot encode
one_hot_cols = train.select_dtypes(exclude='number').columns.drop(high_cardinality).to_list()
one_hot_cols

['BOROUGH',
 'NEIGHBORHOOD',
 'TAX_CLASS_AT_PRESENT',
 'BUILDING_CLASS_AT_PRESENT',
 'BUILDING_CLASS_AT_TIME_OF_SALE']

In [207]:
X_test

,BOROUGH,NEIGHBORHOOD,TAX_CLASS_AT_PRESENT,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE
18235,2,OTHER,1,5913,878,A1,10471.0,1.0,0.0,1.0,2272.0,1930.0,1,A1
18239,2,OTHER,1,5488,48,A2,10465.0,1.0,0.0,1.0,720.0,1935.0,1,A2
18244,3,OTHER,1,5936,31,A1,11209.0,1.0,0.0,1.0,2210.0,1925.0,1,A1
18280,3,OTHER,1,7813,24,A5,11210.0,1.0,0.0,1.0,1520.0,1915.0,1,A5
18285,3,OTHER,1,8831,160,A9,11229.0,1.0,0.0,1.0,840.0,1925.0,1,A9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23029,4,OTHER,1,13215,3,A2,11422.0,1.0,0.0,1.0,1478.0,1925.0,1,A2
23031,4,OTHER,1,11612,73,A1,11420.0,1.0,0.0,1.0,1280.0,1930.0,1,A1
23032,4,OTHER,1,11808,50,A0,11420.0,1.0,0.0,1.0,1333.0,1945.0,1,A0
23033,4,OTHER,1,12295,23,A1,11434.0,1.0,0.0,1.0,1020.0,1935.0,1,A1


In [0]:
import category_encoders as ce
encoder = ce.OneHotEncoder(cols=one_hot_cols,use_cat_names=True)
X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)

In [209]:
print(X_train.shape)
print(X_test.shape)

(2507, 47)
(644, 47)


In [178]:
print(X_test.columns)
X_train.columns

Index(['BOROUGH_2', 'BOROUGH_3', 'BOROUGH_4', 'BOROUGH_5', 'BOROUGH_1',
       'NEIGHBORHOOD_OTHER', 'NEIGHBORHOOD_FOREST HILLS',
       'NEIGHBORHOOD_EAST NEW YORK', 'NEIGHBORHOOD_FLUSHING-NORTH',
       'NEIGHBORHOOD_BOROUGH PARK', 'NEIGHBORHOOD_BEDFORD STUYVESANT',
       'NEIGHBORHOOD_ASTORIA', 'TAX_CLASS_AT_PRESENT_1',
       'TAX_CLASS_AT_PRESENT_1D', 'BLOCK', 'LOT',
       'BUILDING_CLASS_AT_PRESENT_A1', 'BUILDING_CLASS_AT_PRESENT_A2',
       'BUILDING_CLASS_AT_PRESENT_A5', 'BUILDING_CLASS_AT_PRESENT_A9',
       'BUILDING_CLASS_AT_PRESENT_A8', 'BUILDING_CLASS_AT_PRESENT_A0',
       'BUILDING_CLASS_AT_PRESENT_B2', 'BUILDING_CLASS_AT_PRESENT_S1',
       'BUILDING_CLASS_AT_PRESENT_A6', 'BUILDING_CLASS_AT_PRESENT_A4',
       'BUILDING_CLASS_AT_PRESENT_A3', 'ZIP_CODE', 'RESIDENTIAL_UNITS',
       'COMMERCIAL_UNITS', 'TOTAL_UNITS', 'GROSS_SQUARE_FEET', 'YEAR_BUILT',
       'TAX_CLASS_AT_TIME_OF_SALE', 'BUILDING_CLASS_AT_TIME_OF_SALE_A1',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A2',
   

Index(['BOROUGH_3', 'BOROUGH_4', 'BOROUGH_2', 'BOROUGH_5', 'BOROUGH_1',
       'NEIGHBORHOOD_OTHER', 'NEIGHBORHOOD_FLUSHING-NORTH',
       'NEIGHBORHOOD_EAST NEW YORK', 'NEIGHBORHOOD_BEDFORD STUYVESANT',
       'NEIGHBORHOOD_FOREST HILLS', 'NEIGHBORHOOD_BOROUGH PARK',
       'NEIGHBORHOOD_ASTORIA', 'TAX_CLASS_AT_PRESENT_1',
       'TAX_CLASS_AT_PRESENT_1D', 'BLOCK', 'LOT',
       'BUILDING_CLASS_AT_PRESENT_A9', 'BUILDING_CLASS_AT_PRESENT_A1',
       'BUILDING_CLASS_AT_PRESENT_A5', 'BUILDING_CLASS_AT_PRESENT_A0',
       'BUILDING_CLASS_AT_PRESENT_A2', 'BUILDING_CLASS_AT_PRESENT_A3',
       'BUILDING_CLASS_AT_PRESENT_S1', 'BUILDING_CLASS_AT_PRESENT_A4',
       'BUILDING_CLASS_AT_PRESENT_A6', 'BUILDING_CLASS_AT_PRESENT_A8',
       'BUILDING_CLASS_AT_PRESENT_B2', 'BUILDING_CLASS_AT_PRESENT_S0',
       'BUILDING_CLASS_AT_PRESENT_B3', 'ZIP_CODE', 'RESIDENTIAL_UNITS',
       'COMMERCIAL_UNITS', 'TOTAL_UNITS', 'GROSS_SQUARE_FEET', 'YEAR_BUILT',
       'TAX_CLASS_AT_TIME_OF_SALE', 'BUILDING_CLA

## Using Select KBest

In [322]:
from sklearn.feature_selection import f_regression, SelectKBest

#Select some number of features that best correlates with target

num_features = 40

selector = SelectKBest(score_func=f_regression,k=num_features)
X_train_selected = selector.fit_transform(X_train,y_train)
X_test_selected = selector.transform(X_test)


print(type(X_train_selected))
X_train_selected.shape

<class 'numpy.ndarray'>


/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning:

divide by zero encountered in true_divide

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning:

invalid value encountered in true_divide

/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning:

invalid value encountered in greater

/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning:

invalid value encountered in less

/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning:

invalid value encountered in less_equal



(2507, 40)

##Linear Regression

##Ridge Regression

In [331]:
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error


for alpha in [10**1, 10**2, 10**3, 10**4, 10**5, 10**6]:
  #First Scale Data for Ridge Regression
  scaler = StandardScaler()

  X_train_scaled = scaler.fit_transform(X_train_selected)
  X_test_scaled = scaler.transform(X_test_selected)

  #Fit Ridge Regression Model
  model = Ridge(alpha=alpha)

  model.fit(X_train_selected,y_train)
  y_pred = model.predict(X_test_selected)

  mae = mean_absolute_error(y_test,y_pred)
  print(f'alpha = {alpha}:\nMAE = {mae}')

alpha = 10:
MAE = 157896.68182587632
alpha = 100:
MAE = 161565.2714215895
alpha = 1000:
MAE = 164142.8500257787
alpha = 10000:
MAE = 166391.37668257224
alpha = 100000:
MAE = 166877.2730195893
alpha = 1000000:
MAE = 166888.83653605697
